In [2]:
import os
import streamlit as st
import pickle
import time
import langchain
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings  
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader
from langchain.vectorstores import FAISS





In [1]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Access the OpenAI API key
GOOGLE_GEMINI_API_KEY = os.getenv('GOOGLE_GEMINI_API_KEY')
 

In [ ]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",  
    temperature=0,          
    max_tokens=None,           
    api_key=os.getenv("GOOGLE_GEMINI_API_KEY")  
)

response = llm.invoke("What is the capital of France?")
print(response.content)


Paris


# (1) Load data

In [ ]:
loaders = UnstructuredURLLoader(urls=[
    "https://www.moneycontrol.com/news/business/markets/wall-street-rises-as-tesla-soars-on-ai-optimism-11351111.html",
    "https://www.moneycontrol.com/news/business/tata-motors-launches-punch-icng-price-starts-at-rs-7-1-lakh-11098751.html"
])
data = loaders.load() 
len(data)

2

# (2) Split data to create chunks

In [16]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

# As data is of type documents we can directly use split_documents over split_text in order to get the chunks.
docs = text_splitter.split_documents(data)

In [17]:
len(docs)

59

In [23]:
print(docs[0])

page_content='English

Hindi

Gujarati

Specials

Trending Stocks

Waaree Energies INE377N01017, WAAREEENER, 544277

Eternal INE758T01015, ETERNAL, 543320

Ola Electric INE0LXG01040, OLAELEC, 544225

Suzlon Energy INE040H01021, SUZLON, 532667

BSE Limited INE118H01025, BSE, 0



Quotes

Mutual Funds

Commodities

Futures & Options

Currency

News

Cryptocurrency

Forum

Notices

Videos

Glossary

All

Hello, Login Hello, LoginLog-inor Sign-UpMy AccountMy Profile My PortfolioMy WatchlistMy AlertsMy MessagesPrice AlertsMy Profile My PROMy PortfolioMy WatchlistMy AlertsMy MessagesPrice AlertsLogoutLoans up to ₹50 LAKHSFixed DepositsCredit CardsLifetime Free Credit ScoreChat with UsDownload AppFollow us on:

Go Ad-Free

My Alerts' metadata={'source': 'https://www.moneycontrol.com/news/business/markets/wall-street-rises-as-tesla-soars-on-ai-optimism-11351111.html'}


# (3) Create embeddings for these chunks and save them to FAISS index

In [47]:
# Create the embeddings of the chunks using 
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004",google_api_key=GOOGLE_GEMINI_API_KEY)

# Pass the documents and embeddings inorder to create FAISS vector index
vectorindex_gemini = FAISS.from_documents(docs,embeddings)

In [45]:
vectorindex_gemini.save_local("vector_index") 

In [52]:
x = FAISS.load_local("vector_index", embeddings,allow_dangerous_deserialization=True)

retriever = x.as_retriever()

# (4) Retrieve similar embeddings for a given question and call LLM to retrieve final answer

In [55]:
chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=retriever)
chain

RetrievalQAWithSourcesChain(verbose=False, combine_documents_chain=MapReduceDocumentsChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Use the following portion of a long document to see if any of the text is relevant to answer the question. \nReturn any relevant text verbatim.\n{context}\nQuestion: {question}\nRelevant text, if any:'), llm=ChatGoogleGenerativeAI(model='models/gemini-2.0-flash', google_api_key=SecretStr('**********'), temperature=0.0, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x0000028571630B50>, default_metadata=()), output_parser=StrOutputParser(), llm_kwargs={}), reduce_documents_chain=ReduceDocumentsChain(verbose=False, combine_documents_chain=StuffDocumentsChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question', 'summaries'], input_

In [56]:
query = "what is the price of Tiago iCNG?"
# query = "what are the main features of punch iCNG?"

langchain.debug=True

chain({"question": query}, return_only_outputs=True)

C:\Users\Shreyansh Negi\AppData\Local\Temp\ipykernel_1488\377114326.py:6: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain({"question": query}, return_only_outputs=True)


[chain/start] [chain:RetrievalQAWithSourcesChain] Entering Chain run with input:
{
  "question": "what is the price of Tiago iCNG?"
}
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "input_list": [
    {
      "context": "The company also said it has also introduced the twin-cylinder technology on its Tiago and Tigor models.\n\nThe Tiago iCNG is priced between Rs 6.55 lakh and Rs 8.1 lakh, while the Tigor iCNG comes at a price range of Rs 7.8 lakh to Rs 8.95 lakh.\n\nTata Motors Passenger Vehicles Ltd Head-Marketing, Vinay Pant said these introductions put together will make the company's CNG line up \"appealing, holistic, and stronger than ever\".\n\nPTI\n\nfirst published: Aug 4, 2023 02:17 pm\n\nBusiness News,\n\nBudget 2025 News,\n\nSensex, and\n\nNifty updates. Obtain\n\nPerso

{'answer': 'The Tiago iCNG is priced between Rs 6.55 lakh and Rs 8.1 lakh.\n',
 'sources': 'https://www.moneycontrol.com/news/business/tata-motors-launches-punch-icng-price-starts-at-rs-7-1-lakh-11098751.html'}